##Face Mask Detection by Denys Sikorskyi

**Downloading data**

Firstly, we create two folder on Google Drive: train and validation. Both of them are also divided in two folders: with and without. Performing rescaling, rotation, zoom, flip and shift range for training folder. Performing only rescaling for validation.

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
train_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Face Mask Detection/train'
val_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Face Mask Detection/validation'
IMG_HEIGHT, IMG_WIDTH = 224, 224
batch_size = 16
epochs = 5
train_image_generator = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                ) 
validation_image_generator = ImageDataGenerator(rescale=1./255)
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=val_path,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
Found 5609 images belonging to 2 classes.
Found 1944 images belonging to 2 classes.


**Creating model**

Importing InceptionV3 model for implemetentantion. In first session, we freeze inception layers and train top layers. Then, we freeze top layers and train inception layers. 

In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
model.summary()
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 2)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.0000001)
callbacks = [earlystop,learning_rate_reduction]
# train the model on the new data for a few epochs
history = model.fit(train_data_gen, 
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=1944//batch_size,
    steps_per_epoch=5609//batch_size,
    callbacks=callbacks)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
model.summary()

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit(train_data_gen, 
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=1944//batch_size,
    steps_per_epoch=5609//batch_size,
    callbacks=callbacks)


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_188 (Conv2D)            (None, None, None,   864         ['input_3[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_188 (Batch  (None, None, None,   96         ['conv2d_188[0][0]']             
 Normalization)                 32)                                                         

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


350/350 [==============================] - 105s 288ms/step - loss: 0.2123 - accuracy: 0.9573 - val_loss: 0.2122 - val_accuracy: 0.9483 - lr: 0.0010
Epoch 2/5
350/350 [==============================] - 98s 279ms/step - loss: 0.0668 - accuracy: 0.9862 - val_loss: 0.1113 - val_accuracy: 0.9768 - lr: 0.0010
Epoch 3/5
350/350 [==============================] - 97s 278ms/step - loss: 0.0556 - accuracy: 0.9878 - val_loss: 0.0634 - val_accuracy: 0.9804 - lr: 0.0010
Epoch 4/5
350/350 [==============================] - 96s 275ms/step - loss: 0.0442 - accuracy: 0.9893 - val_loss: 0.0799 - val_accuracy: 0.9824 - lr: 0.0010
Epoch 5/5
350/350 [==============================] - 97s 277ms/step - loss: 0.0446 - accuracy: 0.9920 - val_loss: 0.0639 - val_accuracy: 0.9850 - lr: 0.0010
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/5
350/350 [==============================] - 160s 290ms/step - loss: 0.0701 - accuracy: 0.9741 - val_loss: 0.0909 - val_accuracy: 0.9711 - lr: 1.0000e-04
Epoch 2/5
350/350 [==============================] - 99s 284ms/step - loss: 0.0508 - accuracy: 0.9839 - val_loss: 0.0727 - val_accuracy: 0.9783 - lr: 1.0000e-04
Epoch 3/5
350/350 [==============================] - 100s 285ms/step - loss: 0.0297 - accuracy: 0.9898 - val_loss: 0.0679 - val_accuracy: 0.9793 - lr: 1.0000e-04
Epoch 4/5
350/350 [==============================] - 100s 286ms/step - loss: 0.0293 - accuracy: 0.9896 - val_loss: 0.0637 - val_accuracy: 0.9804 - lr: 1.0000e-04
Epoch 5/5
350/350 [==============================] - 100s 285ms/step - loss: 0.0274 - accuracy: 0.9914 - val_loss: 0.0563 - val_accuracy: 0.9814 - lr: 1.0000e-04


:In the end, we can get accuracy about 99% on train dataset and 98 - 98.5% on the validation.